In [4]:
import praw, random, pandas as pd, time
import datetime
import random, os

In [5]:
fantasy_mmorpg_words =['abomination', 'abyss', 'acolyte', 'adventure', 'adventurer', 'alchemy', 'alien', 'alignment', 
                       'alliance', 'altar', 'amulet', 'ancient', 'angel', 'apocalypse', 'archer', 'archmage', 'artifact', 
                       'assassin', 'assault', 'avatar', 'axe', 'barbarian', 'bard', 'battleground', 'battlefield', 'beast', 
                       'beholder', 'berserker', 'blessing', 'blight', 'blood', 'bounty', 'bow', 'campaign', 'captain', 'castle', 
                       'catacomb', 'chalice', 'champion', 'charm', 'chaos', 'chart', 'cleric', 'companion', 'conquest', 'covenant', 'crusade', 
                       'crypt', 'crystal', 'cult', 'curse', 'dagger', 'dark', 'death', 'defender', 'deity', 'demon', 'destiny', 'dimension', 
                       'divine', 'doom', 'dragon', 'dragonborn', 'druid', 'dungeon', 'duel', 'eldritch', 'elemental', 'elementalist', 'elf', 
                       'elixir', 'emissary', 'empire', 'encounter', 'enchantment', 'epic', 'eternal', 'eternity', 'exile', 
                       'experience', 'explorer', 'fate', 'fey', 'fiend', 'fireball', 'flame', 'forge', 'fortress', 'frost', 'fury', 
                       'galactic', 'galaxy', 'game','game master', 'gate', 'gates', 'gem', 'ghost', 'ghoul', 'giant', 'gladiator', 'glory', 'gnoll', 'gnome', 
                       'goblin', 'golem', 'grave', 'guardian', 'guild', 'halfling', 'harbinger', 'healer', 'hellfire', 'hero', 'heroic', 'holy', 
                       'horde', 'horror', 'hymn', 'icon', 'immortal', 'incantation', 'incarnation',
                       'invocation', 'jinn', 'journey', 'knight', 'kraken', 'lair', 'lance', 'legend', 'legendary', 'leviathan', 
                       'life', 'life drain', 'lich', 'light', 'longsword', 'lore', 'lunar', 'lycan', 'lycanthrope', 'mage', 'magic', 'magma', 
                       'maiden', 'manticore', 'maul', 'medieval', 'medusa', 'mercenary', 'meteor', 'mystic', 'mystical', 'myth', 'nemesis', 
                       'nightmare', 'ninja', 'nymph','oracle', 'orc', 'paladin', 'pantheon', 'paragon', 'path', 'phoenix', 'planet', 
                       'plague', 'pledge', 'portal', 'possession', 'primal', 'prophecy', 'protection', 'prototype','quicksilver', 
                       'rage', 'ranger', 'reaper', 'rebirth', 'recon', 'redemption', 'relic', 'revelation', 'rewards','revenant', 
                       'revolution', 'rogue', 'rune', 'sacrifice', 'sage', 'scepter', 'scroll', 'seer', 'serpent', 'shadow', 
                       'shadowblade', 'shaman', 'shade', 'shapeshifter', 'shield', 'ship', 'shroud', 'siren', 'slaughter', 'slayer', 'sorcerer', 
                       'sorcery', 'sphinx', 'spire', 'spirit', 'spiritual', 'stalker', 'star', 'stygian', 'summon', 'summoner', 'sunlight', 
                       'supernatural', 'supernova', 'sword', 'swordsman', 'talisman', 'tavern', 'temple', 'tome', 'torment', 'tour', 
                       'transformation', 'transit', 'trap', 'treasure', 'tribe', 'triumph', 'troll', 'twilight', 'unicorn', 
                       'unity', 'vampire', 'vanguard', 'vengeance', 'venom', 'victory', 'vortex', 'warp', 'warrior', 'warden', 'werewolf', 
                       'wizard', 'wrath', 'wretched', 'wyrm', 'wyvern', 'XP']


unique_items = []
for item in fantasy_mmorpg_words:
    if item.lower() not in unique_items:
        unique_items.append(item.lower())
fantasy_mmorpg_words = unique_items

In [6]:
reddit = praw.Reddit(client_id='h61xEBGhbqO6A92hi2AB5g',client_secret='yPtlrpTaIt3jgOwYkVQtY1NhV1UzbA', user_agent='Reddit_Scrape')

In [ ]:
columns = ["title", "subreddit", "score", "url", "created_utc", "num_comments", "content"]
df = pd.DataFrame(columns=columns)

# Define parameters for the keyword search
num_keywords = 3          # Number of random keywords to select for each search
post_limits = 100         # Maximum number of posts to retrieve per search
iterations = 15           # Number of iterations (i.e., number of searches)

df = pd.DataFrame(columns=columns)

csv_file = 'reddit_posts.csv'

if not os.path.exists(csv_file):
    df.to_csv('reddit_posts.csv')

# Function to search Reddit posts with random keywords
def search_reddit_with_keywords(keywords_list, num_keywords, limit):
    global df  # Use the global DataFrame

    # Loop to perform searches for the specified number of iterations

        # Select random keywords from the provided list
    random_keywords = random.choices(keywords_list, k=num_keywords)
    print(f"Recherche avec les mots-clés aléatoires : {random_keywords}")
        
        # Construct a search query string with AND logic for all selected keywords
    search_query = ' AND '.join(random_keywords)
        
        # Perform the search on Reddit
    for submission in reddit.subreddit('all').search(search_query, limit=limit):
            # Check if the post is already in the DataFrame based on its URL
        if submission.url not in df['url'].values:
                # Format the created_utc timestamp to D/M/Y format
            created_date = datetime.datetime.utcfromtimestamp(submission.created_utc).strftime('%d/%m/%Y')
                # Create a temporary dictionary with the new search result
            temp_dict = {
                    "title": submission.title,
                    "subreddit": str(submission.subreddit),
                    "score": submission.score,
                    "url": submission.url,
                    "created_utc": created_date,
                    "num_comments": submission.num_comments,
                    "content": submission.selftext
                }
                # Create a temporary DataFrame and append it to the main DataFrame
            temp_df = pd.DataFrame([temp_dict])
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

for i in range(iterations):
    print(f"Itération {i+1}")
    # Créer un DataFrame vide pour stocker les résultats
    result = pd.DataFrame(columns=columns)
    
    # Effectuer la recherche Reddit avec les mots-clés
    results_df = search_reddit_with_keywords(fantasy_mmorpg_words, num_keywords=num_keywords, limit=post_limits)
    
    # Concaténer les nouveaux résultats avec les résultats précédents
    results = pd.concat([results, results_df], ignore_index=True)
    
    # Supprimer les doublons basés sur l'URL pour assurer l'unicité
    results.drop_duplicates(subset=['url'], keep='first', inplace=True)
    
    # Enregistrer les résultats dans le fichier CSV
    if i == 1:
        results.to_csv(csv_file, index=False, encoding='utf-8')
    else:
        results.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8')
    results.shape    
print(f"Les résultats ont été enregistrés dans {csv_file}")

In [17]:
results.shape

(6925, 7)

In [18]:
results.head()

,title,subreddit,score,url,created_utc,num_comments,content
0,"The Crimson Moon's Vengeance is Justified, but...",Genshin_Lore,40,https://www.reddit.com/r/Genshin_Lore/comments...,15/05/2024,49,"Alright, first off I Read a post by u/Shiroudr..."
1,I used AI to create my character backstory and...,BaldursGate3,0,https://www.reddit.com/r/BaldursGate3/comments...,16/07/2023,4,**Chat GPT prompt** \n\nWrite a compelling bac...
2,The Omniarch (Lorepost),wizardposting,13,https://www.reddit.com/r/wizardposting/comment...,19/02/2024,133,"**1,000 years ago**\n\n[The Omniarch, Avatar o..."
3,"My idea for a universe, need help building ple...",worldbuilding,1,https://www.reddit.com/r/worldbuilding/comment...,10/06/2023,8,I have a basic plot of my MC. She is a wife of...
4,"What shaders would be good for this, if I do n...",DestinyFashion,1,https://www.reddit.com/r/DestinyFashion/commen...,22/10/2023,5,I really don't want this Hunter set to be bori...
